encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
quantize_lut = True
# for method in [METHOD_MITHRAL, METHOD_PQ]:

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"

auto_train_change_nbits = False # 是否根据已运行的训练性能结果改变nbits自动训练，（train_sam_num取已训练的最大值）
auto_train_change_upcast = False # 是否根据已运行的训练性能结果改变upcast自动训练，（train_sam_num取已训练的最大值）

if auto_train_change_nbits:
    nbits_trained = 8
if auto_train_change_upcast:
    upcast_trained = 16
nbits_goal = 12
upcast_goal = -1
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数
upcast_every = upcast_goal # 要运行的upcast

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train_change_nbits and not auto_train_change_upcast:
    ncodebooks = 512 # max:512
    ncentroids = 16
    train_sam_num = 50 # 训练集样本数
elif auto_train_change_nbits:
    param2change = "nbits"
    param_trained = nbits_trained
    param_goal = nbits_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])
elif auto_train_change_upcast:
    param2change = "upcast_every"
    param_trained = upcast_trained
    param_goal = upcast_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

     cb   ct  n_train_sam
0    16   16         1000
1    16   64          500
2    16  128          500
4    16  256          250
5    32   16         1000
6    32   64          575
10   32  128          250
11   32  256          150
13   64   16         1000
16   64   64          350
19   64  128          200
20   64  256           75
21  128   16          625
27  128   64          125
28  128  128           50
29  128  256           20
30  256   16          100
31  256   64           75
33  512   64           20


In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])


In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        upcast_every=upcast_every, bias_path=AMM_train_dirs["biaspath"])
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every)

running method:  Mithral
X.shape:  (51200, 512)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    7.183620929718018
learn_multisplits(): returning loss:  0.02316792931742384
learn_multisplits(): initial loss:    1.7999025583267212
learn_multisplits(): returning loss:  0.004802439591003349
learn_multisplits(): initial loss:    1.0954980850219727
learn_multisplits(): returning loss:  2.4915350422816118e-05
learn_multisplits(): initial loss:    8.47905445098877
learn_multisplits(): returning loss:  0.016014074351915042
learn_multisplits(): initial loss:    0.4075913727283478
learn_multisplits(): returning loss:  0.0010110686724971174
learn_multisplits(): initial loss:    100.24736785888672
learn_multisplits(): returning loss:  0.37082407996058464
learn_multisplits(): initial loss:    5911.62109375
learn_multisplits(): returning loss:  25.29250192642212
learn_multisplits(): initial loss:    0.02854217030107975
learn_multisplits(): returning loss:  0
learn_mul

/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../clusterize.py:783: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 254. / upper_val


learn_multisplits(): initial loss:    11.052423477172852
learn_multisplits(): returning loss:  0.06046868089833879
learn_multisplits(): initial loss:    5450.103515625
learn_multisplits(): returning loss:  22.207971334457397
learn_multisplits(): initial loss:    37.017608642578125
learn_multisplits(): returning loss:  0.19271257385844365
learn_multisplits(): initial loss:    1.249909520149231
learn_multisplits(): returning loss:  0.0014240499804145657
learn_multisplits(): initial loss:    67.55850219726562
learn_multisplits(): returning loss:  1.2937266454100609
learn_multisplits(): initial loss:    234.5479736328125
learn_multisplits(): returning loss:  3.1853117742575705
learn_multisplits(): initial loss:    18.321269989013672
learn_multisplits(): returning loss:  0.06079623629921116
learn_multisplits(): initial loss:    17.46610450744629
learn_multisplits(): returning loss:  0.047358763880765764
learn_multisplits(): initial loss:    0.432903915643692
learn_multisplits(): returning l

In [7]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PQ or method == METHOD_PLUTO or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[ 0.57623553  0.01797867  0.5628867  ...  0.3004758   0.04877434
   0.5656202 ]
 [ 0.5244714   0.03946567  0.5247961  ...  0.32977626  0.05463443
   0.52459955]
 [ 0.5615853   0.01309526  0.5531199  ...  0.29363903  0.03412411
   0.5607368 ]
 ...
 [ 0.44536018  0.12053026  0.2435118  ...  0.26433858  0.19332322
   0.2882426 ]
 [ 0.37308574 -0.12559354  0.14584363 ...  0.33172962  0.08686493
   0.3741906 ]
 [ 0.36624897  0.0863464   0.22495484 ...  0.07095561  0.14448914
   0.24429192]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
